In [1]:
import os

In [2]:
os.chdir("/home/tanush-reddy/workspace/Wine-Grade-Estimator")

In [3]:
%pwd

'/home/tanush-reddy/workspace/Wine-Grade-Estimator'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path 
    train_data_path: Path 
    test_data_path: Path
    model_name: str
    alpha: float 
    l1_ratio: float
    target_column: str

In [5]:
from src.constants import *
from src.utils.common import read_yaml, create_directories


In [6]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH, schema_file_path=SCHEMA_FILE_PATH):
        self.config_file_path = config_file_path
        self.params_file_path = params_file_path
        self.schema_file_path = schema_file_path

        self.config = read_yaml(self.config_file_path)
        self.params = read_yaml(self.params_file_path)
        self.schema = read_yaml(self.schema_file_path)

        create_directories([self.config.artifacts_root])
        
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([Path(config.root_dir)])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            train_data_path=Path(config.train_data_path),
            test_data_path=Path(config.test_data_path),
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        
        return model_trainer_config
        
        
    

In [10]:
import os
from src import logger
from src.exception import CustomException
import sys
import joblib
from sklearn.linear_model import ElasticNet
import pandas as pd

In [12]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        try:
            logger.info("Training the model...")
            train_df = pd.read_csv(self.config.train_data_path)
            test_df = pd.read_csv(self.config.test_data_path)
            
            X_train = train_df.drop(columns=[self.config.target_column],axis=1)
            y_train = train_df[self.config.target_column]
            X_test = test_df.drop(columns=[self.config.target_column],axis=1)
            y_test = test_df[self.config.target_column]
            
            lr= ElasticNet(
                alpha=self.config.alpha,
                l1_ratio=self.config.l1_ratio,
                random_state=42
            )
            
            lr.fit(X_train, y_train)
            joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
            
            logger.info(f"Model trained and saved at {self.config.root_dir / self.config.model_name}")
        except Exception as e:
            raise CustomException(e, sys) from e

In [ ]:
try:
    config_manager = ConfigurationManager()
    model_trainer_config = config_manager.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e: 
    logger.error(f"Error occurred during model training: {e}")
    raise CustomException(e, sys) from e

[2025-07-29 23:36:25,145] 28 WineGradeEstimator - INFO - YAML file config/config.yaml read successfully.
[2025-07-29 23:36:25,147] 28 WineGradeEstimator - INFO - YAML file params.yaml read successfully.
[2025-07-29 23:36:25,148] 28 WineGradeEstimator - INFO - YAML file schema.yaml read successfully.
[2025-07-29 23:36:25,149] 46 WineGradeEstimator - INFO - Directory created: artifacts
[2025-07-29 23:36:25,149] 46 WineGradeEstimator - INFO - Directory created: artifacts/model_trainer
[2025-07-29 23:36:25,150] 7 WineGradeEstimator - INFO - Training the model...
[2025-07-29 23:36:25,160] 25 WineGradeEstimator - INFO - Model trained and saved at artifacts/model_trainer/model.joblib
